<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/0_Station_Choice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Set-up
from google.colab import auth, files
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
import pandas as pd
pd.set_option('display.max_columns', None)

Authenticated


In [7]:
# Get the station and crime data

client = bigquery.Client(project='assignment-305921')

stations = client.query('''
SELECT usaf, wban, name, country, state, call, lat, lon, elev, 
cast(CONCAT(SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.begin, 0, 4), "-", SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.begin, 5, 2), "-", SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.begin, 7, 2))as DATE) as firstDate,
cast(CONCAT(SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.end, 0, 4), "-", SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.end, 5, 2), "-", SUBSTRING(`bigquery-public-data.noaa_gsod.stations`.end, 7, 2))as DATE) as lastDate
FROM `bigquery-public-data.noaa_gsod.stations`
WHERE state = 'IL'
ORDER BY usaf
''').to_dataframe()
# I've reformatted the 'begin' and 'end' columns so that they can be processed as dates and also renamed them from SQL keywords.

crime_2016 = client.query('''
SELECT *
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year = 2016
ORDER BY date
''').to_dataframe()

In [8]:
# Filter the list of stations to only those with data between 2016 - 2020
# The printed list of lat,longs can be pasted into https://mobisoftinfotech.com/tools/plot-multiple-points-on-map/ to see their locations

filtered_start = stations[stations.firstDate <= pd.to_datetime('2016-01-01')]
filtered_end =  filtered_start[filtered_start.lastDate >= pd.to_datetime('2020-12-31')]
print(filtered_end[['lat','lon']].to_csv(header=None, index=False, line_terminator=',\n'))


41.425,-88.419,
37.186,-88.751,
39.016,-87.65,
38.607,-87.727,
38.089,-88.123,
41.932,-88.708,
40.2,-87.6,
42.246,-89.582,
40.933,-90.433,
41.54,-87.532,
41.071,-87.846,
41.352,-89.153,
40.52,-90.652,
40.924,-88.624,
39.7,-87.669,
41.893,-89.078,
40.293,-88.142,
42.046,-90.108,
39.163,-89.675,
38.65,-88.967,
38.323,-88.858,
37.78,-89.25,
38.55,-89.85,
37.75,-89.0,
39.845,-89.684,
38.883,-90.05,
40.483,-88.95,
39.937,-91.192,
37.064,-89.219,
41.96,-87.932,
41.914,-88.246,
38.571,-90.157,
40.04,-88.278,
39.834,-88.866,
39.478,-88.28,
40.668,-89.684,
41.743,-89.676,
41.786,-87.752,
38.764,-87.606,
41.5,-88.167,
42.417,-87.867,
41.604,-88.085,
42.193,-89.093,
41.465,-90.523,
37.811,-88.549,
38.149,-89.699,
41.77,-88.481,
38.379,-88.413,
38.515,-89.092,
38.665,-88.453,
38.722,-88.176,
39.534,-89.328,
39.639,-90.778,
42.121,-87.905,
39.78,-90.238,
40.158,-89.335,
42.0,-87.5,
42.35,-87.82,
41.856,-87.609,
40.053,-88.373,
41.843,-88.851,
39.07,-88.533,
41.019,-89.386,



/usr/local/lib/python3.7/dist-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [9]:
# Filter the crime lat / long to a sample fraction
# The printed list can be pasted into https://mobisoftinfotech.com/tools/plot-multiple-points-on-map/ to see their locations

print(crime_2016.sample(frac = 0.005)[['latitude','longitude']].to_csv(header=None, index=False, line_terminator=',\n'))

41.93725566,-87.728151373,
41.957909084,-87.735570155,
41.685591052,-87.535263257,
41.907180257,-87.628780592,
41.89761933,-87.761616701,
41.999682102,-87.681753468,
41.898332998,-87.627463695,
41.921606937,-87.665565006,
41.748198383,-87.628176326,
41.793528842,-87.613168806,
41.883500187,-87.627876698,
41.888675607,-87.74687756,
41.910866602,-87.710090197,
41.902158558,-87.764430411,
41.754454943,-87.606525164,
41.735033879,-87.545782092,
41.883500187,-87.627876698,
41.883131184,-87.677672969,
41.832898122,-87.716923673,
41.897230664,-87.721862775,
41.880751096,-87.723855981,
41.936006373,-87.736939803,
41.769434364,-87.589537569,
41.78677329,-87.619023222,
42.006503588,-87.676838156,
41.908417822,-87.67740693,
41.707058457,-87.631779244,
41.877107021,-87.68385564,
41.866286618,-87.715725607,
41.952731382,-87.650553579,
41.967477901,-87.739224006,
41.699723577,-87.699939331,
41.898472898,-87.767177583,
41.757630995,-87.585708249,
41.871025414,-87.699510379,
41.946048865,-87.655587315

In [10]:
# The relevant stations by lat / long are:
print(filtered_end.loc[(filtered_end['lat'] == 41.54) & (filtered_end['lon'] == -87.532)]) # Lansing Municipal
print(filtered_end.loc[(filtered_end['lat'] == 41.96) & (filtered_end['lon'] == -87.932)]) # O'Hare International
print(filtered_end.loc[(filtered_end['lat'] == 41.786) & (filtered_end['lon'] == -87.752)]) # Midway International
print(filtered_end.loc[(filtered_end['lat'] == 41.976) & (filtered_end['lon'] == -87.648)]) # Foster Avenue
print(filtered_end.loc[(filtered_end['lat'] == 41.856) & (filtered_end['lon'] == -87.609)]) # Northerly Island

      usaf   wban                  name country state  call    lat     lon  \
21  722126  04879  LANSING MUNICIPAL AP      US    IL  KIGQ  41.54 -87.532   

       elev   firstDate    lastDate  
21  +0187.8  2006-01-01  2021-09-22  
      usaf   wban                          name country state  call    lat  \
64  725300  94846  CHICAGO O'HARE INTERNATIONAL      US    IL  KORD  41.96   

       lon     elev   firstDate    lastDate  
64 -87.932  +0201.8  1946-10-01  2021-09-22  
      usaf   wban                      name country state  call     lat  \
87  725340  14819  CHICAGO MIDWAY INTL ARPT      US    IL  KMDW  41.786   

       lon     elev   firstDate    lastDate  
87 -87.752  +0186.5  1973-01-01  2021-09-22  
Empty DataFrame
Columns: [usaf, wban, name, country, state, call, lat, lon, elev, firstDate, lastDate]
Index: []
       usaf   wban              name country state  call     lat     lon  \
129  998499  99999  NORTHERLY ISLAND      US    IL  None  41.856 -87.609   

        e

By name these stations are:
* Lansing Municipal
* O'Hare International
* Midway International
* Foster Avenue
* Northerly Island

In [11]:
# Get the Weather data
client = bigquery.Client(project='assignment-305921')

weather = []

for year in range(2016, 2021):
  weather.append(client.query('''
  SELECT * 
  FROM `bigquery-public-data.noaa_gsod.gsod{}`
  WHERE (stn = '722126' AND wban = '04879') or (stn = '725300' AND wban = '94846') or (stn = '725340' AND wban = '14819') or (stn = '998497' AND wban = '99999') or (stn = '998499' AND wban = '99999')
  ORDER BY stn
  '''.format(year)).to_dataframe())

In [12]:
# Count data from each station in each year
stns = ['722126', '725300', '725340', '998497', '998499']
index = 0
for year in weather:
  print(2016 + index)
  for stn in stns:
    print(stn, year.loc[year['stn']==stn, 'stn'].agg(['count']).to_string())
  print('-----')
  index += 1

2016
722126 count    366
725300 count    366
725340 count    366
998497 count    365
998499 count    363
-----
2017
722126 count    365
725300 count    365
725340 count    365
998497 count    361
998499 count    249
-----
2018
722126 count    365
725300 count    365
725340 count    365
998497 count    115
998499 count    115
-----
2019
722126 count    365
725300 count    365
725340 count    365
998497 count    250
998499 count    350
-----
2020
722126 count    366
725300 count    366
725340 count    366
998497 count    64
998499 count    353
-----


Foster Avenue and Northerly Island are missing significant amouts of data. I have decided not to use these stations.
The 3 stations being used are:
* Lansing Municipal
* O'Hare International
* Midway International
